In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from tqdm import tqdm
import sys
sys.path.append('../')

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Dropout, InputLayer, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
import tensorflow_addons as tfa

from sklearn.preprocessing import MinMaxScaler, RobustScaler, StandardScaler, normalize
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix, classification_report

from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, classification_report

from utils.data_utils import mod_df, drop_features, inverse_mod_X, inverse_mod_y, apply_savgol_filter, apply_median_filter, apply_maximum_filter, apply_is_zero, apply_standard_scale
from utils.visualize_utils import plot_history, evaluate_model
from my_model import create_model_1, create_model_1_1, create_model_1_2, create_model_1_3, create_model_1_4, create_model_2, create_model_2_1, create_model_2_2, create_model_3


tf.__version__

In [ ]:
train_df = pd.read_csv(os.path.join('data', 'unionTrain.csv'))
test_df = pd.read_csv(os.path.join('data', 'unionTest.csv'))

In [ ]:
X_train, y_train = mod_df(train_df)
X_test, y_test = mod_df(test_df)

In [ ]:
def feature_engineering(df):
    result = df.copy()
    
    # select & drop features
    result = result.drop(['id', 'timestep'], axis=1)
    # result = drop_features(result,[0,1,2,5,15,16,17,18,21,20,23,24])
    # result = drop_features(result, [15, 16, 17, 18, 20, 21, 23, 24])
    # result = result[['1Y','2Y','4Y','5Y','6Y','7X','8Y','11Y','12Y','13Y','14Y','20X','22Y','23X','24X','24Y']] # ts = 0.07
    result = result[['1Y', '2Y', '4X', '4Y', '5Y', '6Y', '7X', '8Y', '10X', '11Y', '12Y', '13Y', '14X', '14Y', '20X', '21Y', '22Y', '23X', '24X', '24Y']] # ts = 0.04
    # result = result[['1Y', '2Y', '3Y', '4X', '4Y', '5Y', '6Y', '7X', '7Y', '8Y', '10X', '11Y', '12X', '12Y', '13Y', '14X', '14Y', '16Y', '20X', '21Y', '22X', '22Y', '23X', '23Y', '24X', '24Y']] # ts = 0.001
    DEFAULT_FEATURE = result.columns.to_list()
    
    # add features
    # result['test_0X'] = result['0X']
    FEATURE_COLUMNS = result.columns.to_list()
    for col in tqdm(FEATURE_COLUMNS):
        feature = result[col]
        feature = feature.to_numpy()
        # result[f'{col}_savgol_1'] = apply_savgol_filter(feature, window_size=21, polynomial=1)
        # result[f'{col}_savgol_7'] = apply_savgol_filter(feature, window_size=21, polynomial=2)
        # result[f'{col}_savgol_2'] = apply_savgol_filter(feature, window_size=21, polynomial=3)
        # result[f'{col}_savgol_3'] = apply_savgol_filter(feature, window_size=21, polynomial=5)
        # result[f'{col}_savgol_4'] = apply_savgol_filter(feature, window_size=11, polynomial=1)
        # result[f'{col}_savgol_8'] = apply_savgol_filter(feature, window_size=11, polynomial=2)
        # result[f'{col}_savgol_5'] = apply_savgol_filter(feature, window_size=11, polynomial=3)
        # result[f'{col}_savgol_6'] = apply_savgol_filter(feature, window_size=11, polynomial=5)
        # result[f'{col}_median'] = apply_median_filter(feature)
        # result[f'{col}_max'] = apply_maximum_filter(feature, window_size=5)
        # result[f'{col}_sav_med'] = apply_median_filter(apply_savgol_filter(feature, window_size=21), window_size=5)
        # result[f'{col}_sav_min_max_scale'] = apply_min_max_scale(apply_savgol_filter(feature, window_size=11, polynomial=2))
        # result[f'{col}_standard_min_max_scale'] = apply_min_max_scale(apply_standard_scale(feature))
        

        # result[f'{col}_max'] = apply_max_value(feature)
        # result[f'{col}_min'] = apply_min_value(feature)
        # result[f'{col}_mean'] = apply_mean(feature)
        # result[f'{col}_std'] = apply_std(feature)
        # result[f'{col}_is_zero'] = apply_is_zero(feature)
        # result[f'{col}_min_max_scale'] = apply_min_max_scale(feature)
        result[f'{col}_standard_scale'] = apply_standard_scale(feature)
    
    # modify features
    FEATURE_COLUMNS = result.columns.to_list()
    for col in tqdm(FEATURE_COLUMNS):
        feature = result[col]
        feature = feature.to_numpy()
        # result[col] = apply_savgol_filter(feature)
        # result[col] = apply_median_filter(feature)
        # result[col] = apply_median_filter(apply_savgol_filter(feature, window_size=21), window_size=5)
    
    # drop default features
    result = result.drop(DEFAULT_FEATURE, axis=1)
      
    return result

X_train_1 = feature_engineering(X_train)
X_test_1 = feature_engineering(X_test)

In [ ]:
def get_scaler(scaler_name='MinMaxScaler'):
    if scaler_name == 'RobustScaler':
        scaler = RobustScaler()
    elif scaler_name == 'MinMaxScaler':
        scaler = MinMaxScaler()
    else:
        scaler = None
    return scaler
# scaler_name = 'RobustScaler'
scaler_name = 'MinMaxScaler'
# scaler_name = None
scaler = get_scaler(scaler_name)

In [ ]:
if scaler:
  X_train_scaled = scaler.fit_transform(X_train_1)
  X_test_scaled = scaler.transform(X_test_1)
else:
  X_train_scaled, X_test_scaled = X_train_1.to_numpy(), X_test_1.to_numpy()

In [ ]:
# joblib.dump(scaler, os.path.join('saved_scaler', f'{scaler_name}_wo_max.pkl'))

In [ ]:
X_train_modified, y_train_modified = inverse_mod_X(X_train_scaled), inverse_mod_y(y_train)
X_test_modified, y_test_modified = inverse_mod_X(X_test_scaled), inverse_mod_y(y_test)
print(X_train_modified.shape, y_train_modified.shape)

In [ ]:
input_shape, n_output = (X_train_modified.shape[-2], X_train_modified.shape[-1]), y_train_modified.shape[1]
def build_model():
  # model = create_model_3(input_shape,
  #   n_output,
  #   head_size=32,
  #   num_heads=2,
  #   ff_dim=4,
  #   num_transformer_blocks=4,
  #   mlp_units=[64],
  #   dropout=0.25,
  #   mlp_dropout=0.25
  #   )
  # model = create_model_1(input_shape, n_output)
  # model = create_model_1_0(input_shape, n_output)
  # model = create_model_1_2(input_shape, n_output)
  # model = create_model_1_3(input_shape, n_output)
  # model = create_model_1_4(input_shape, n_output)
  # model = create_model_1_5(input_shape, n_output)
  # model = create_model_2(input_shape, n_output)
  # model = create_model_2_1(input_shape, n_output)
  # model = Sequential([
  #     InputLayer(input_shape),
  #     Bidirectional(LSTM(64, return_sequences=True)),
  #     Bidirectional(LSTM(32, return_sequences=False)),
  #     Dense(25, activation= 'selu'),
  #     Dense(n_output, activation="softmax")
  # ])
  model = Sequential([
        InputLayer(input_shape),
        Conv1D(filters=64,
               kernel_size=8,
               strides=1,
               activation='relu',
               padding='same'),
        MaxPooling1D(pool_size=4),
        Bidirectional(LSTM(64, return_sequences=False)),
        Dense(8, activation='relu'),
        Dense(n_output, activation="softmax")
    ])
  return model
tmp = build_model()
tmp.summary()

In [ ]:
optimizer = Adam(learning_rate=1e-3)
reduce_lr = ReduceLROnPlateau(
                                monitor='val_loss',
                                factor=0.5,
                                patience=10,
                                min_lr=1e-4
                              )
early_stopping = EarlyStopping(
                                monitor='loss',
                                min_delta=1e-3,
                                patience=12,
                                restore_best_weights=True
                              )

In [ ]:
def train(epochs=30, batch_size=32, verbose=0):
  model = build_model()
  model.compile(
                optimizer=optimizer,
                loss=CategoricalCrossentropy(),
                metrics=[
                    'accuracy',
                    tfa.metrics.F1Score(y_train_modified.shape[-1])
                ]
              )
  history = model.fit(X_train_modified, y_train_modified,
            batch_size=batch_size,
            epochs=epochs,
            shuffle=True,
            verbose=verbose,
            validation_split=0.2,
            callbacks=[
                reduce_lr,
                early_stopping,
                ]
            )
  return model, history

In [ ]:
model_list, history_list = [], []
train_report_list, test_report_list = [], []
n_attempt = 5
for i in range(n_attempt):
    print("Attempt:", i+1)
    model, history = train(epochs=150, batch_size=16, verbose=0)
    model_list.append(model)
    history_list.append(history)
    plot_history(history, show_accuracy=False, validate=True)

    report_train_metrics = evaluate_model(
        model, X_train_modified, y_train_modified)
    train_report_list.append(report_train_metrics)
    print(
        f"train macro avg f1-score: {report_train_metrics['macro avg']['f1-score']}")
    report_test_metrics = evaluate_model(
        model, X_test_modified, y_test_modified)
    test_report_list.append(report_test_metrics)
    print(
        f"test macro avg f1-score: {report_test_metrics['macro avg']['f1-score']}")
    print("-----------------------------------------------------")


In [ ]:
train_macro_f1, test_macro_f1 = [], []
for e in train_report_list:
  train_macro_f1.append(e['macro avg']['f1-score'])
for e in test_report_list:
  test_macro_f1.append(e['macro avg']['f1-score'])
train_macro_f1, test_macro_f1 = np.array(train_macro_f1), np.array(test_macro_f1)

print(f"number of attempt: {n_attempt}")
print(f"avg macro train: {np.mean(train_macro_f1)}, SD: {np.std(train_macro_f1)}")
print(f"min: {np.min(train_macro_f1)}, max: {np.max(train_macro_f1)}")
print(f"avg macro test: {np.mean(test_macro_f1)}, SD: {np.std(test_macro_f1)}")
print(f"min: {np.min(test_macro_f1)}, max: {np.max(test_macro_f1)}")

In [ ]:
# select model
index = 2
model = model_list[index]
history = history_list[index]

In [ ]:
train_predict = model.predict(X_train_modified, batch_size=32)
train_predict = np.argmax(train_predict, axis=1)+1
train_real = np.argmax(y_train_modified, axis=1)+1

# for i in range(len(y_train_modified)):
#     print(f"Index:{i}, Predict:{train_predict[i]}, Real:{train_real[i]}")

In [ ]:
f1_train = f1_score(train_real, train_predict)
accuracy_train = accuracy_score(train_real, train_predict)
# print(f"f1: {f1_train:.4f}\naccuracy: {accuracy_train:.4f}")
print(classification_report(train_real, train_predict, digits=4))
print("---------------------------------------------------------")
sns.heatmap(confusion_matrix(train_real, train_predict),annot = True,fmt = '2.0f')
print()

In [ ]:
test_predict_0 = model.predict(X_test_modified)
test_predict = np.argmax(test_predict_0, axis=1)+1
test_real = np.argmax(y_test_modified, axis=1)+1

# for i in range(len(y_test)):
#     print(f"Index:{i}, Predict:{test_real[i]}, Real:{test_real[i]}")

In [ ]:
f1_test = f1_score(test_real, test_predict)
accuracy_test = accuracy_score(test_real, test_predict)
# print(f"f1: {f1_test:.4f}\naccuracy: {accuracy_test:.4f}")
print(classification_report(test_real, test_predict, digits=4))
print("---------------------------------------------------------")
sns.heatmap(confusion_matrix(test_real, test_predict),annot = True,fmt = '2.0f')
print()